## Project 3: What NYC age groups make the most 911 calls?
by Sophia Cain

# Introduction

**Datasets**

For this project, I will be using two different data sets from [NYC Open Data](https://opendata.cityofnewyork.us/) to explore the volume of 911 calls in New York in relation to age groups. The first data set is [NYPD Calls for Service](https://data.cityofnewyork.us/Public-Safety/NYPD-Calls-for-Service-Year-to-Date-/n2zq-pubd/about_data) which provides the date and borough of 911 calls made in New York. The second data set is [Projected Population 2010-2040 - Total By Age Groups](https://data.cityofnewyork.us/City-Government/Projected-Population-2010-2040-Total-By-Age-Groups/97pn-acdf/about_data) which gives population age data, broken down into 18 age cohorts, by NYC borough. 

**Analysis Question**

What NYC boroughs have the highest concentration of residents above 65 (the age to qualify for medicare)?

What NYC boroughs have the highest concentration of 911 calls?

Is there overlap in the density of residents over 65 and 911 calls made?

**Columns that will (likely) be used**

**Hypothesis**

I hypothesize that there in boroughs with a larger elderly population (65+), there will be a higher volume of 911 calls. 

## Step 1: Load in Data

I start by loading in the necessary packages to complete my analysis. Initially, this is pandas and then the given code to make sure any visulatizations will display correctly on my github site.

I then load in the new data frames that I will be using to conduct this analysis. For the NYC population age data, I load it in as a CSV because the data frame is relativley small. For the NYC 911 data, this is a considerably larger data frame that includes geographical codes. Therefore, I chose to load this in through an API and then convern the JSON data into a data frame.

I use .head to view both data frames and get an idea of what columns and data types I will be working with and need to clean/transform.

In [13]:
import plotly.io as pio

pio.renderers.default = "notebook_connected+plotly_mimetype"

In [14]:
import pandas as pd

NYC_pop_age = pd.read_csv("NYC_pop_age.csv")

NYC_pop_age.head()

,Borough,Age,2010,2015,2020,2025,2030,2035,2040
0,NYC Total,0-4,"521,990","535,209","545,778","547,336","542,426","540,523","546,426"
1,NYC Total,15-19,"539,844","505,783","492,532","519,298","535,024","546,062","546,750"
2,NYC Total,20-24,"647,483","646,075","606,203","591,683","625,253","643,728","657,403"
3,NYC Total,25-29,"736,105","770,396","763,956","715,824","698,195","740,437","762,757"
4,NYC Total,30-34,"667,657","707,726","743,916","740,268","693,684","675,497","715,486"


In [15]:
NYC_pop_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Borough  114 non-null    object
 1   Age      114 non-null    object
 2   2010     114 non-null    object
 3   2015     114 non-null    object
 4   2020     114 non-null    object
 5   2025     114 non-null    object
 6   2030     114 non-null    object
 7   2035     114 non-null    object
 8   2040     114 non-null    object
dtypes: object(9)
memory usage: 8.1+ KB


In [16]:
NYC_pop_age["Age"].unique()

array(['0-4', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44',
       '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79',
       '80-84', '85+', 'Total', '5-9', '10-14'], dtype=object)

From this, I can see that all of the data is stored as objects. I also used the .unique function to see how many age ranges there are, so I can combine all of the 65+ age ranges.

In [17]:
NYC_pop_age.tail()

,Borough,Age,2010,2015,2020,2025,2030,2035,2040
109,Staten Island,10-14,"30,797","30,703","30,307","30,001","30,541","31,074","30,932"
110,Queens,10-14,"124,320","123,672","129,351","130,384","132,038","133,473","133,813"
111,Manhattan,10-14,"58,229","55,455","60,331","62,902","66,322","66,700","63,630"
112,Brooklyn,10-14,"159,404","155,314","166,761","174,242","175,992","173,419","171,265"
113,Bronx,10-14,"99,159","92,423","95,703","100,081","104,065","105,297","105,010"


I use the .tail function because the .head function only gave me the first rows that showed the population totals, but I need to see how the Boroughs are displayed

In [18]:
import requests

response = requests.get("https://data.cityofnewyork.us/resource/n2zq-pubd.json")
NYC_911 = response.json()

# convert to a data frame

pd.DataFrame(NYC_911)

,cad_evnt_id,create_date,incident_date,incident_time,nypd_pct_cd,boro_nm,patrl_boro_nm,geo_cd_x,geo_cd_y,radio_code,typ_desc,cip_jobs,add_ts,disp_ts,arrivd_ts,closng_ts,latitude,longitude
0,108642792,2025-01-01T00:00:00.000,2024-12-31T00:00:00.000,23:22:32,116,(null),(null),1056625,180527,53S,VEHICLE ACCIDENT: SPECIAL CONDITION,Non CIP,2025-01-01T00:44:37.000,2025-01-01T00:45:19.000,2025-01-01T01:21:05.000,2025-01-01T01:21:07.000,40.661894046000043,-73.739133148999940
1,108643213,2025-01-01T00:00:00.000,2024-12-31T00:00:00.000,23:58:23,52,BRONX,PATROL BORO BRONX,1013953,255677,54I2,AMBULANCE CASE: INJURY/OUTSIDE,Non CIP,2025-01-01T00:17:38.000,2025-01-01T00:19:56.000,2025-01-01T00:20:50.000,2025-01-01T00:39:14.000,40.868407122000065,-73.892607676999944
2,108643225,2025-01-01T00:00:00.000,2025-01-01T00:00:00.000,00:00:08,84,BROOKLYN,PATROL BORO BKLYN NORTH,987351,193828,75S,STATION INSPECTION BY TRANSIT BUREAU PERSONNEL,Non CIP,2025-01-01T00:00:08.000,2025-01-01T00:00:08.000,2025-01-01T00:00:08.000,2025-01-01T00:42:04.000,40.698697527000036,-73.988818053999978
3,108643226,2025-01-01T00:00:00.000,2025-01-01T00:00:00.000,00:00:43,70,BROOKLYN,PATROL BORO BKLYN SOUTH,993152,168419,50G2,DISORDERLY: GROUP/OUTSIDE,Non CIP,2025-01-01T00:00:44.000,2025-01-01T00:38:50.000,2025-01-01T01:35:33.000,2025-01-01T02:06:57.000,40.628951512000071,-73.967930976999980
4,108643227,2025-01-01T00:00:00.000,2025-01-01T00:00:00.000,00:00:49,13,MANHATTAN,PATROL BORO MAN SOUTH,987653,210944,10H1,INVESTIGATE/POSSIBLE CRIME: CALLS FOR HELP/INSIDE,Non CIP,2025-01-01T00:00:49.000,2025-01-01T00:13:25.000,NaN,2025-01-01T00:19:14.000,40.745676707000030,-73.987720274999958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,108644339,2025-01-01T00:00:00.000,2025-01-01T00:00:00.000,01:17:14,47,BRONX,PATROL BORO BRONX,1024141,265555,10V2,INVESTIGATE/POSSIBLE CRIME: SUSP VEHICLE/OUTSIDE,Non CIP,2025-01-01T01:17:14.000,2025-01-01T01:17:28.000,2025-01-01T01:18:49.000,2025-01-01T01:24:17.000,40.895478963000073,-73.855713308999952
996,108644340,2025-01-01T00:00:00.000,2025-01-01T00:00:00.000,01:17:17,88,BROOKLYN,PATROL BORO BKLYN NORTH,990501,188891,54E9,AMBULANCE CASE: EDP/TRANSIT,Non CIP,2025-01-01T01:17:17.000,2025-01-01T01:17:17.000,2025-01-01T01:17:17.000,2025-01-01T02:08:56.000,40.685144935000039,-73.977462530999958
997,108644341,2025-01-01T00:00:00.000,2025-01-01T00:00:00.000,01:17:23,49,BRONX,PATROL BORO BRONX,1020860,251010,75S,STATION INSPECTION BY TRANSIT BUREAU PERSONNEL,Non CIP,2025-01-01T01:17:23.000,2025-01-01T01:17:23.000,2025-01-01T01:17:23.000,2025-01-01T02:12:03.000,40.855571684000040,-73.867660344999933
998,108644342,2025-01-01T00:00:00.000,2025-01-01T00:00:00.000,01:17:33,108,QUEENS,PATROL BORO QUEENS NORTH,1000828,212736,75M,TRAIN RUN/MOBILE ORDER MAINTENANCE SWEEP,Non CIP,2025-01-01T01:17:33.000,2025-01-01T01:17:33.000,2025-01-01T01:17:33.000,2025-01-01T01:29:44.000,40.750580413000080,-73.940168275999952


## Step 2: Clean Data

I am now going to clean this data to select only the variables that I need to work with and make sure that it is usable for my later visualizations.

In [19]:
# First, I will filter for borough to get rid of the rows with population totals and I will select for the year 2025 because it is the most recent data.

NYC_pop_age_columns_keep = ["Borough", "Age", "2025"]

NYC_pop_age_2025 = NYC_pop_age[NYC_pop_age_columns_keep]

boroughs = ["Staten Island", "Queens", "Manhattan", "Brooklyn", "Bronx"]

NYC_pop_age_2025 = NYC_pop_age_2025[NYC_pop_age_2025["Borough"].isin(boroughs)]

NYC_pop_age_2025.head()

,Borough,Age,2025
17,Bronx,0-4,"111,127"
18,Bronx,15-19,"103,114"
19,Bronx,20-24,"107,423"
20,Bronx,25-29,"117,887"
21,Bronx,30-34,"122,434"


In [29]:
# Now I will combine the Age rows into three groups: child (0-19), adult (20-64), and senior (65+).

import numpy as np


# I am converting the 2025 population column from an object to an integer and removing the commas so it will add nicely when I combine age groups.
NYC_pop_age_2025["2025"] = NYC_pop_age_2025["2025"].str.replace(",", "").astype(int)

# I am making the three popualtion groups from the age ranges provided.
child = ["0-4", "15-19"]

adult = [
    "20-24",
    "25-29",
    "30-34",
    "35-39",
    "40-44",
    "45-49",
    "50-54",
    "55-59",
    "60-64",
]

senior = ["65-69", "70-74", "75-79", "80-84", "85+"]

# I am making a new variable with all three ranges to convert the rows later.
age_groups = child + adult + senior

# Here is where I am seleting the rows as the three age groups. After recieved an error, I added the default='unknown' to account for data that was not in the three groups.
NYC_pop_age_2025["Age_Group"] = np.select(
    [
        NYC_pop_age_2025["Age"].isin(child),
        NYC_pop_age_2025["Age"].isin(adult),
        NYC_pop_age_2025["Age"].isin(senior),
    ],
    ["child", "adult", "senior"],
    default="unknown",
)

# I am using group by to combine the total of the age group rows by borough.
NYC_pop_age_2025_grouped = NYC_pop_age_2025.groupby(
    ["Borough", "Age_Group"], as_index=False
)["2025"].sum()

# I am viewing the cleaned data.
NYC_pop_age_2025_grouped.head()

AttributeError: Can only use .str accessor with string values!

In [21]:
NYC_pop_age_2025_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Borough    20 non-null     object
 1   Age_Group  20 non-null     object
 2   2025       20 non-null     object
dtypes: object(3)
memory usage: 612.0+ bytes


## Conlusion

counfounding variables?

higher populations? more traffic?